<a href="https://colab.research.google.com/github/LeonBecken/Bezkrovnyi_Assignments/blob/main/Bezkrovnyi_7_1_1_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 7.1

> Replace all TODOs with your code.
>
> Do not change any other code and do not add/remove cells!

In [ ]:
# Do not edit this cell

from typing import List, Dict

## Clients

### Task 1

Write a function that takes a GitHub repository name as input and uses the [GitHub Repository API](https://docs.github.com/en/rest/repos) to retrieve and return information about the specified repository, including:

* Repository Name
* Owner
* Description
* License (if available)
* Creation Date

Use the `requests` module for this task.

In [1]:
from typing import Dict, List
import requests

HEADERS = {"Accept": "application/vnd.github+json"}

def _ensure_ok(response: requests.Response) -> None:

    if 200 <= response.status_code < 300:
        return
    msg = ""
    try:
        msg = response.json().get("message", "")
    except ValueError:
        msg = response.text[:200]
    raise RuntimeError(f"HTTP {response.status_code}: {msg or 'Request failed'}")

def _get_json(url: str, timeout: int = 10):

    try:
        resp = requests.get(url, headers=HEADERS, timeout=timeout)
    except requests.RequestException as e:
        raise RuntimeError(f"Network error: {e}") from e

    _ensure_ok(resp)

    try:
        return resp.json()
    except ValueError as e:
        raise ValueError(f"Invalid JSON from {url}: {e}") from e


def get_repository_info(repo_name: str) -> Dict[str, str]:

    if repo_name.count("/") != 1:
        raise ValueError("Invalid repo_name format. Use 'owner/repo'.")

    repo_name = repo_name.strip()
    url = f"https://api.github.com/repos/LeonBecken/Bezkrovnyi_Assignments"

    data = _get_json(url)

    return {
        "Repository Name": data.get("name") or "N/A",
        "Owner": (data.get("owner") or {}).get("login") or "N/A",
        "Description": data.get("description") or "No description",
        "License": ((data.get("license") or {}) or {}).get("name") or "No license",
        "Creation Date": data.get("created_at") or "N/A",
    }


def get_recent_commits(repo_name: str) -> List[Dict[str, str]]:

    if repo_name.count("/") != 1:
        raise ValueError("Invalid repo_name format. Use 'owner/repo'.")

    repo_name = repo_name.strip()
    url = f"https://api.github.com/repos/LeonBecken/Bezkrovnyi_Assignments/commits?per_page=5"

    items = _get_json(url)
    result: List[Dict[str, str]] = []

    for item in items if isinstance(items, list) else []:
        commit_block = item.get("commit", {}) or {}
        author_block = commit_block.get("author", {}) or {}
        result.append(
            {
                "Commit Message": commit_block.get("message") or "N/A",
                "Author": author_block.get("name") or "N/A",
                "Date": author_block.get("date") or "N/A",
                "Link": item.get("html_url") or "N/A",
            }
        )
    return result

### Task 2

Write a function that takes a GitHub repository name and uses the [GitHub Commits API](https://docs.github.com/en/rest/commits/commits) to retrieve and return information about the five most recent commits in a selected GitHub repository, including:
* Commit Message
* Author
* Date
* Link to the commit on GitHub

Use the `requests` module for this task.

In [2]:
from typing import List, Dict
import requests

HEADERS = {"Accept": "application/vnd.github+json"}


def _ensure_ok(response: requests.Response) -> None:

    if 200 <= response.status_code < 300:
        return
    try:
        msg = response.json().get("message", "")
    except ValueError:
        msg = response.text[:200]
    raise RuntimeError(f"HTTP {response.status_code}: {msg or 'Request failed'}")


def _get_json(url: str, timeout: int = 10):

    try:
        resp = requests.get(url, headers=HEADERS, timeout=timeout)
    except requests.RequestException as e:
        raise RuntimeError(f"Network error: {e}") from e

    _ensure_ok(resp)

    try:
        return resp.json()
    except ValueError as e:
        raise ValueError(f"Invalid JSON from {url}: {e}") from e


def get_recent_commits(repo_name: str) -> List[Dict[str, str]]:

    if repo_name.count("/") != 1:
        raise ValueError("Invalid repo_name format. Use 'owner/repo'.")

    repo_name = repo_name.strip()
    url = f"https://api.github.com/repos/Bezkrovnyi_Assignments/commits?per_page=5"

    items = _get_json(url)
    result: List[Dict[str, str]] = []

    if isinstance(items, list):
        for item in items:
            commit_block = item.get("commit", {}) or {}
            author_block = commit_block.get("author", {}) or {}
            result.append(
                {
                    "Commit Message": commit_block.get("message") or "N/A",
                    "Author": author_block.get("name") or "N/A",
                    "Date": author_block.get("date") or "N/A",
                    "Link": item.get("html_url") or "N/A",
                }
            )

    return result